# 单臂重建 (Fix版)

目标：把 FLIPPING_v3 的单臂数据点云和 URDF mesh 对齐。

本notebook将 `pose_in_link` 同时按两种解释计算：
- 解释A：`pose_in_link` = base_to_camera（相机在base下的pose）
- 解释B：`pose_in_link` = camera_to_base（base在相机下的pose）

你可以通过开关选择哪一种，方便快速验证。


In [3]:
import os
import cv2
import h5py
import numpy as np
import open3d as o3d
import trimesh
import k3d
from omegaconf import OmegaConf
from scipy.spatial.transform import Rotation

from airexo.helpers.urdf_robot import forward_kinematic
from airexo.helpers.constants import ROBOT_PREDEFINED_TRANSFORMATION, O3D_RENDER_TRANSFORMATION

# ===================== 配置 =====================
SCENE_PATH = '/data/haoxiang/data/FLIPPING_v3/train/scene_0001'
LOWDIM_H5_PATH = os.path.join(SCENE_PATH, 'lowdim/lowdim.h5')
FRAME_IDX = 0
CAM_SERIAL = '104122060902'
RGB_PATH = f'{SCENE_PATH}/cam_{CAM_SERIAL}/color/1767593840262.png'
DEPTH_PATH = f'{SCENE_PATH}/cam_{CAM_SERIAL}/depth/1767593840262.png'

# JSON里的标定数据（你给的 pose_in_link + intrinsics）
CALIB_DATA = {
    'pose_in_link': [
        0.07783932332093665,
        0.2078814260418823,
        0.34723683952957585,
        0.2273133855008057,
        -0.6785647482083789,
        0.6637673415778982,
        -0.21746591345696367,
    ],
    'intrinsics': [
        [915.384521484375, 0.0, 633.3715209960938],
        [0.0, 914.9421997070312, 354.1505432128906],
        [0.0, 0.0, 1.0],
    ],
}

# pose_in_link 上下文（你贴的 JSON）：
# { is_global: true, parent_link_name: 'world', pose_in_link: [x,y,z,qx,qy,qz,qw] }
# 含义应理解为：相机在 parent_link_name(world) 坐标系下的位姿，即 T_world_cam（world -> camera）。
# 渲染/对齐时，我们需要的是 cam_to_base，其中 base 在本 notebook 里等同于 world（因为点云本身就在相机坐标系下生成）。
# 所以应使用：cam_to_base = inv(T_world_cam) = T_cam_world。
# 若仍不对齐，最常见原因是 JSON 的 pose 不是 optical frame，而深度反投影使用的是 optical frame。
# 这里增加一个开关：是否在外参后乘一个 Optical Frame 修正（ROS常见约定）。
#   optical_correction=False：cam_to_base = inv(T_world_cam)
#   optical_correction=True ：cam_to_base = O3D_RENDER_TRANSFORMATION @ inv(T_world_cam)
# 注意：这是用于快速验证轴向约定差异的工程手段。
OPTICAL_CORRECTION = False

# ===================== 工具函数 =====================
def pose7d_to_T(pose_7d):
    pose_7d = np.asarray(pose_7d, dtype=np.float64)
    t = pose_7d[:3]
    q = pose_7d[3:]  # [qx,qy,qz,qw]
    R = Rotation.from_quat(q).as_matrix()
    T = np.eye(4)
    T[:3,:3] = R
    T[:3,3] = t
    return T

def make_cam_to_base(pose_in_link, meaning):
    """
    返回: (cam_to_base, base_to_camera)

    - meaning == 'world_to_camera': pose_in_link 表示 T_world_cam (world->camera)
      那么 cam_to_base(此处base=world) = T_cam_world = inv(T_world_cam)
"""
    T = pose7d_to_T(pose_in_link)
    if meaning == 'world_to_camera':
        T_world_cam = T
        T_cam_world = np.linalg.inv(T_world_cam)
        # 可选 optical frame 轴修正：如果 pose_in_link 对应 camera_link 而深度反投影是 optical frame，
        # 常见需要额外乘一个固定轴变换。这里用 O3D_RENDER_TRANSFORMATION 做快速验证。
        if 'OPTICAL_CORRECTION' in globals() and OPTICAL_CORRECTION:
            T_cam_world = O3D_RENDER_TRANSFORMATION @ T_cam_world
        return T_cam_world.astype(np.float32), T_world_cam.astype(np.float32)
    else:
        raise ValueError('meaning must be world_to_camera')

# ===================== 1) 读取关节数据 =====================
print('📂 加载数据...')
with h5py.File(LOWDIM_H5_PATH, 'r') as f:
    joint_positions = f['joint_position_rad_062046'][FRAME_IDX]  # (7,)
    ee_state = f['ee_state_062046'][FRAME_IDX, 0]  # 夹爪
left_joint = np.concatenate([joint_positions, [ee_state]]).astype(np.float32)
right_joint = np.zeros(8, dtype=np.float32)
right_joint[7] = 0.085
print('左臂关节:', left_joint)

# ===================== 2) 标定矩阵 =====================
intrinsic = np.array(CALIB_DATA['intrinsics'], dtype=np.float32)
cam_to_base, base_to_camera = make_cam_to_base(CALIB_DATA['pose_in_link'], POSE_MEANING)
print('\nPOSE_MEANING =', POSE_MEANING)
print('base_to_camera:')
print(base_to_camera)
print('\ncam_to_base:')
print(cam_to_base)
print('\n验证(base_to_camera @ cam_to_base) 应接近 I:')
print(base_to_camera @ cam_to_base)

# ===================== 3) FK =====================
left_cfg = OmegaConf.load('airexo/configs/joint/left/robot.yaml')
right_cfg = OmegaConf.load('airexo/configs/joint/right/robot.yaml')
print('\n🤖 执行前向运动学...')
cur_transforms, visuals_map = forward_kinematic(
    left_joint=left_joint,
    right_joint=right_joint,
    left_joint_cfgs=left_cfg,
    right_joint_cfgs=right_cfg,
    is_rad=True,
    urdf_file='airexo/urdf_models/robot/robot_inhand.urdf',
    with_visuals_map=True
)
print('✓ FK 完成:', len(cur_transforms), '个 link')

# ===================== 4) 点云（相机系） =====================
print('\n☁️ 生成点云...')
color_img = cv2.cvtColor(cv2.imread(RGB_PATH), cv2.COLOR_BGR2RGB)
depth_img = cv2.imread(DEPTH_PATH, cv2.IMREAD_UNCHANGED).astype(np.float32)
fx, fy = float(intrinsic[0,0]), float(intrinsic[1,1])
cx, cy = float(intrinsic[0,2]), float(intrinsic[1,2])
h, w = depth_img.shape
intrinsic_o3d = o3d.camera.PinholeCameraIntrinsic(width=int(w), height=int(h), fx=fx, fy=fy, cx=cx, cy=cy)
depth_o3d = o3d.geometry.Image(depth_img.astype(np.uint16))
color_o3d = o3d.geometry.Image(color_img.astype(np.uint8))
rgbd = o3d.geometry.RGBDImage.create_from_color_and_depth(color_o3d, depth_o3d, depth_scale=1000.0, convert_rgb_to_intensity=False)
pcd_cam = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd, intrinsic_o3d)
# 与mesh一致：都应用 O3D_RENDER_TRANSFORMATION
pcd_cam.transform(O3D_RENDER_TRANSFORMATION)
print('✓ 点云:', len(pcd_cam.points), '个点')

# ===================== 5) Mesh（相机系） =====================
print('\n🎨 K3D 渲染...')
plot = k3d.plot(background_color=0xFFFFFF)

mesh_count = 0
for link, transform in cur_transforms.items():
    if link not in visuals_map:
        continue
    for v in visuals_map[link]:
        if v.geom_param is None:
            continue
        mesh_path = os.path.join('airexo/urdf_models/robot', v.geom_param)
        if not os.path.exists(mesh_path):
            continue
        mesh = trimesh.load(mesh_path, force='mesh')
        # 变换链（与官方 renderer.py 形式一致）
        tf = O3D_RENDER_TRANSFORMATION @ cam_to_base @ ROBOT_PREDEFINED_TRANSFORMATION @ transform.matrix() @ v.offset.matrix()
        mesh.apply_transform(tf)
        color = 0xff0000 if (link.startswith('L') or link.startswith('left')) else 0xaaaaaa
        plot += k3d.mesh(mesh.vertices.astype(np.float32), mesh.faces.astype(np.uint32), color=color, opacity=0.75)
        mesh_count += 1

# 点云加入K3D
pcd_points = np.asarray(pcd_cam.points).astype(np.float32)
pcd_colors = (np.asarray(pcd_cam.colors) * 255).astype(np.uint32)
pcd_colors_packed = (pcd_colors[:,0] << 16) | (pcd_colors[:,1] << 8) | pcd_colors[:,2]
plot += k3d.points(pcd_points, colors=pcd_colors_packed, point_size=0.002, shader='flat')

# 坐标轴
axis_size = 0.3
plot += k3d.vectors(origins=[[0,0,0],[0,0,0],[0,0,0]], vectors=[[axis_size,0,0],[0,axis_size,0],[0,0,axis_size]], colors=[0xff0000,0x00ff00,0x0000ff], line_width=0.01)

print('总 mesh 数:', mesh_count)
print('✅ 完成！切换 POSE_MEANING 后重新运行该 cell 做对比。')
plot.display()


📂 加载数据...
左臂关节: [ 6.6571248e-01 -1.0406699e+00 -1.9467313e+00  2.1450579e+00
  1.6500233e+00  3.9253008e-01  1.2164255e+00  1.5000001e-04]

POSE_MEANING = world_to_camera
base_to_camera:
[[-0.8020744  -0.01980016  0.5968958   0.07783932]
 [-0.5971872   0.01548308 -0.8019524   0.20788142]
 [ 0.006637   -0.99968404 -0.02424299  0.34723684]
 [ 0.          0.          0.          1.        ]]

cam_to_base:
[[-0.8020744  -0.5971872   0.006637    0.18427245]
 [-0.01980016  0.01548308 -0.99968404  0.34544972]
 [ 0.5968958  -0.8019524  -0.02424299  0.1286671 ]
 [ 0.          0.          0.          1.        ]]

验证(base_to_camera @ cam_to_base) 应接近 I:
[[ 9.9999994e-01 -6.1908999e-08 -5.9525052e-10  7.4505806e-09]
 [-6.1908999e-08  1.0000000e+00  1.2277248e-09  0.0000000e+00]
 [-5.9525052e-10  1.2277248e-09  9.9999994e-01  2.9802322e-08]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  1.0000000e+00]]

🤖 执行前向运动学...
✓ FK 完成: 37 个 link

☁️ 生成点云...
✓ 点云: 823841 个点

🎨 K3D 渲染...


/home/haoxiang/miniforge3/envs/airexo/lib/python3.8/site-packages/traittypes/traittypes.py:98: UserWarning: Given trait value dtype "float32" does not match required type "float32". A coerced copy has been created.
  warnings.warn(
/home/haoxiang/miniforge3/envs/airexo/lib/python3.8/site-packages/traittypes/traittypes.py:98: UserWarning: Given trait value dtype "uint32" does not match required type "uint32". A coerced copy has been created.
  warnings.warn(


总 mesh 数: 35
✅ 完成！切换 POSE_MEANING 后重新运行该 cell 做对比。


Output()